In [1]:
# Huggingface: Qwen/Qwen2-7B-Instruct-GPTQ-Int8
model_qwen2 = r"/mnt/workspace/model/Qwen2-7B-Instruct-GPTQ-Int8"
!ls /mnt/workspace/model/Qwen2-7B-Instruct-GPTQ-Int8

added_tokens.json	model-00001-of-00002.safetensors  tokenizer_config.json
config.json		model-00002-of-00002.safetensors  tokenizer.json
generation_config.json	model.safetensors.index.json	  vocab.json
merges.txt		special_tokens_map.json


In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_qwen2,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_qwen2)

2024-07-02 17:56:27,637 - modelscope - INFO - PyTorch version 2.3.0+cu121 Found.
2024-07-02 17:56:27,640 - modelscope - INFO - TensorFlow version 2.16.1 Found.
2024-07-02 17:56:27,641 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-02 17:56:27,675 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 03830e53bae2c0a8f8ac72c969ac6ac1 and a total number of 980 components indexed
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-02 17:56:29.607969: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
20

In [3]:
prompt = "香蕉是什么颜色的？"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


香蕉通常是黄色的，但也可能是绿色的，具体取决于它们是否成熟。未成熟的香蕉表皮是绿色的，而成熟的香蕉则是黄色的，有时也会带有褐色斑点。在某些品种中，香蕉也可能呈现红色、紫色或带有条纹的颜色。


In [17]:
from transformers import pipeline
from modelscope.utils.constant import Tasks
generator = pipeline(Tasks.text_generation, model=model, tokenizer=tokenizer)
result = generator(messages, max_length=50)[0]['generated_text']
print(result)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '香蕉是什么颜色的？'}, {'role': 'assistant', 'content': '香蕉通常是黄色的，但也可能因成熟度、品种或环境因素而呈现绿色、红色或其他颜色。成熟的香蕉会'}]
